In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
import os
from langchain_openai import ChatOpenAI  # Notez le changement ici

# Définition de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-rPBnsXimJyDjm9m7k8ZxrMdz1Lgs6 JoLeEVAbOl0kiGIZTWPKAtav6nWikZPSewB3oeo055nFcLW9rnMorepgfeAZNQQz5FUUwrtmQsA"

# Création du modèle
llm = ChatOpenAI(
    model="gpt-4o-mini",  # Notez que c'est 'model' et non 'model_name' dans les versions récentes
    temperature=0
)

In [ ]:
active_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate Active Prompting examples based on user questions.
    
    Active Prompting is a specialized technique within Few-Shot Chain-of-Thought (CoT) prompting that:
    1. Starts with training questions/exemplars
    2. Asks the LLM to solve these exemplars
    3. Calculates uncertainty (disagreement in responses)
    4. Identifies exemplars with highest uncertainty
    5. Uses human annotators to rewrite these high-uncertainty exemplars
    6. Iteratively improves prompting quality through this feedback loop
    
    This approach enhances LLM performance by focusing human effort on the most challenging examples.
    
    Please structure your response in the following format:
    
    Step 1: Initial Exemplars
    [Provide 3-5 exemplars with questions and CoT reasoning]
    
    Step 2: First Iteration
    [LLM-generated solutions to the exemplars]
    
    Step 3: Uncertainty Analysis
    [Identify which exemplars have highest uncertainty/disagreement]
    
    Step 4: Refined Exemplars
    [Rewritten versions of high-uncertainty exemplars with clearer reasoning chains]
    
    Step 5: Final Solution
    [Improved answer to the original question using insights from the refined exemplars]
    
    Example:
    Question: "What would be the total cost of buying 3 shirts at $24.99 each and 2 pairs of pants at $39.95 each, with an 8% sales tax?"
    
    Initial Exemplars:
    Exemplar 1: 
    Q: If a book costs $12.50 and I have a 20% off coupon, how much will I pay?
    A: To find the discounted price, I need to calculate 20% of $12.50 and subtract it from the original price.
    20% of $12.50 = 0.20 × $12.50 = $2.50
    Final price = $12.50 - $2.50 = $10.00
    Therefore, I will pay $10.00 for the book.
    
    Exemplar 2:
    Q: What is the total cost of 4 apples at $0.75 each and 3 oranges at $0.60 each?
    A: First, I'll calculate the cost of the apples.
    Cost of apples = 4 × $0.75 = $3.00
    Next, I'll calculate the cost of the oranges.
    Cost of oranges = 3 × $0.60 = $1.80
    Total cost = $3.00 + $1.80 = $4.80
    Therefore, the total cost is $4.80.
    
    Exemplar 3:
    Q: If a meal costs $45.00 and I want to leave a 15% tip, what will my total bill be?
    A: To find the tip amount, I calculate 15% of the meal cost.
    Tip amount = 0.15 × $45.00 = $6.75
    Total bill = Meal cost + Tip = $45.00 + $6.75 = $51.75
    Therefore, my total bill will be $51.75.
    
    First Iteration:
    For our question about shirts and pants:
    Cost of shirts = 3 × $24.99 = $74.97
    Cost of pants = 2 × $39.95 = $79.90
    Subtotal = $74.97 + $79.90 = $154.87
    Tax amount = 0.08 × $154.87 = $12.39
    Total cost = $154.87 + $12.39 = $167.26
    
    Uncertainty Analysis:
    The calculation seems straightforward, but there might be uncertainty in how to handle the cents in the multiplication and whether to round the tax amount. 
     Let's identify this as an area of potential disagreement.
    
    Refined Exemplars:
    Exemplar 2 (Rewritten):
    Q: What is the total cost of 4 apples at $0.75 each and 3 oranges at $0.60 each?
    A: First, I'll calculate the cost of the apples.
    Cost of apples = 4 × $0.75 = $3.00
    Next, I'll calculate the cost of the oranges.
    Cost of oranges = 3 × $0.60 = $1.80
    Subtotal = $3.00 + $1.80 = $4.80
    For retail transactions, we calculate to the nearest cent, so the total cost is exactly $4.80.
    
    Exemplar 3 (Rewritten):
    Q: If a meal costs $45.00 and I want to leave a 15% tip, what will my total bill be?
    A: To find the tip amount, I calculate 15% of the meal cost.
    Tip amount = 0.15 × $45.00 = $6.75
    Total bill = Meal cost + Tip = $45.00 + $6.75 = $51.75
    When calculating percentages, we typically round to the nearest cent, so the total bill is $51.75.
    
    Final Solution:
    To find the total cost of buying 3 shirts and 2 pairs of pants with sales tax:
    
    Step 1: Calculate the cost of the shirts.
    Cost of shirts = 3 × $24.99 = $74.97
    
    Step 2: Calculate the cost of the pants.
    Cost of pants = 2 × $39.95 = $79.90
    
    Step 3: Determine the subtotal before tax.
    Subtotal = $74.97 + $79.90 = $154.87
    
    Step 4: Calculate the sales tax amount.
    Tax amount = 0.08 × $154.87 = $12.3896
    Rounded to the nearest cent: $12.39
    
    Step 5: Compute the final total.
    Total cost = $154.87 + $12.39 = $167.26
    
    Therefore, the total cost would be $167.26.
    """),
    ("human", "{question}"),
    ("assistant", """
    I'll solve this problem using Active Prompting to iteratively improve my approach:
    
    Step 1: Initial Exemplars
    [Provide 3 relevant exemplars with detailed reasoning paths that relate to the problem domain]
    
    Step 2: First Iteration
    [Initial detailed solution attempt with step-by-step reasoning]
    
    Step 3: Uncertainty Analysis
    [Identification of specific steps or concepts where reasoning might be unclear or where multiple approaches could lead to different answers]
    
    Step 4: Refined Exemplars
    [Improved versions of the exemplars that address the identified uncertainties with clearer reasoning chains]
    
    Step 5: Final Solution
    [Comprehensive step-by-step solution that leverages insights from the refined exemplars and addresses all aspects of the original question]
    """)
])

In [19]:
import pandas as pd
import re
from typing import Optional, TypedDict
from langchain.schema import AIMessage, HumanMessage

# Configuration to display the full text in pandas cells
pd.set_option('display.max_colwidth', None)  # Don't truncate columns
pd.set_option('display.width', 1000)         # Generous display width
pd.set_option('display.max_rows', None)      # Display all rows

# Function to generate the Active Prompting prompt and extract the final answer
def generate_active_prompt_node(state):
    question = state['messages'][-1].content  # Get the last question
    prompt_value = active_prompt_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    
    # Keep the full response for processing
    full_response = response.content
    
    # Method 1: Look for the format "Final Answer (Displayed to User)"
    final_answer_pattern = r'# Final Answer \(Displayed to User\)([\s\S]*)'
    final_answer_match = re.search(final_answer_pattern, full_response)
    
    if final_answer_match:
        final_solution = final_answer_match.group(1).strip()
    else:
        # Method 2: Look for Step 5: Final Solution
        final_solution_pattern = r'\*\*Step 5: Final Solution\*\*([\s\S]*)'
        final_solution_match = re.search(final_solution_pattern, full_response)
        
        if final_solution_match:
            final_solution = final_solution_match.group(1).strip()
        else:
            # Fallback method: extract the last portion of text
            lines = full_response.strip().split('\n')
            if len(lines) > 5:
                # Get the last significant lines
                final_solution = '\n'.join(lines[-10:])
            else:
                final_solution = "Final solution not found in the expected format."
    
    # Create a pandas DataFrame for clean display
    result_df = pd.DataFrame({
        'Question': [question],
        'Final Answer': [final_solution]
    })
    
    # Also add the full response
    result_df['Full Processing'] = [full_response]
    
    return {"messages": [response], "result_df": result_df}

# Definition of state with the addition of DataFrame
class State(TypedDict):
    messages: list  # Simplified to avoid problems with Annotated
    result_df: Optional[pd.DataFrame]  # Added DataFrame to store the result

# Graph creation
graph_builder = StateGraph(State)
graph_builder.add_node("generate_active_prompt", generate_active_prompt_node)

# Graph configuration
graph_builder.set_entry_point("generate_active_prompt")
graph_builder.add_edge("generate_active_prompt", END)
graph = graph_builder.compile()

# Usage example
inputs = {"messages": [HumanMessage(content="Marie gave half of her apples to Jean. Jean ate three apples," 
                                         " then gave half of what he had left to Pierre. " 
                                         "Pierre now has 4 apples. How many apples did Marie have at the beginning?")],
          "result_df": None}  # Initialize result_df to None

# Graph execution
final_output = None
for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_active_prompt":
            if "result_df" in value:
                final_output = value["result_df"]

# Display result with multiple methods to ensure a complete output
if final_output is not None:
    print("\n=== FINAL RESULT (SUMMARY) ===\n")
    
    # Display of the summarized result
    print("QUESTION:")
    print(final_output['Question'].iloc[0])
    print("\nFINAL ANSWER:")
    print(final_output['Final Answer'].iloc[0])
    
    print("\n\n=== COMPLETE PROCESSING (ALL STEPS) ===\n")
    print(final_output['Full Processing'].iloc[0])
    
    # Save to a text file (both versions)
    with open('complete_result.txt', 'w', encoding='utf-8') as f:
        f.write("=== FINAL RESULT (SUMMARY) ===\n\n")
        f.write("QUESTION:\n")
        f.write(final_output['Question'].iloc[0])
        f.write("\n\nFINAL ANSWER:\n")
        f.write(final_output['Final Answer'].iloc[0])
        f.write("\n\n\n=== COMPLETE PROCESSING (ALL STEPS) ===\n\n")
        f.write(final_output['Full Processing'].iloc[0])
    
     


=== FINAL RESULT (SUMMARY) ===

QUESTION:
Marie gave half of her apples to Jean. Jean ate three apples, then gave half of what he had left to Pierre. Pierre now has 4 apples. How many apples did Marie have at the beginning?

FINAL ANSWER:
To find out how many apples Marie had initially, we follow these steps:

1. Let M be the number of apples Marie had initially.
2. Marie gave half of her apples to Jean: Jean receives M/2 apples.
3. Jean ate 3 apples: Apples left with Jean = (M/2) - 3.
4. Jean gave half of what he had left to Pierre: Pierre receives [(M/2) - 3]/2 apples.
5. We know Pierre has 4 apples:
   \[
   \frac{(M/2) - 3}{2} = 4
   \]
   Multiplying both sides by 2 gives:
   \[
   (M/2) - 3 = 8
   \]
   Adding 3 to both sides:
   \[
   M/2 = 11
   \]
   Multiplying by 2 gives:
   \[
   M = 22
   \]

Therefore, Marie had **22 apples** at the beginning.


=== COMPLETE PROCESSING (ALL STEPS) ===

**Step 1: Initial Exemplars**

Exemplar 1:  
Q: If Sarah has 10 candies and gives 4 to